 # 单因子分析：规模因子 (Size Factor)

**目标**: 本 Notebook 旨在演示如何使用本框架对一个因子进行完整的回测与分析。

**核心思想**: 我们将遵循《因子投资：方法与实践》中定义的标准因子测试流程。该流程的核心是：
1.  **数据准备**: 加载并清洗干净的股票行情与财务数据。
2.  **因子计算**: 计算我们感兴趣的因子——在这里是规模因子（对数市值）。
3.  **分组回测**: 在每个时间点，根据因子值将所有股票排序并分为N组（例如5组），然后观察每组的未来收益表现。
4.  **业绩分析**: 通过多空组合（做多因子值最高组，做空因子值最低组）的净值曲线、年化收益、夏普比率等指标来评估因子的有效性。

In [ ]:
# -- 步骤 0: 导入必要的库并设置项目路径 --
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

# 设置matplotlib以支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  

# 将项目根目录添加到Python的模块搜索路径中
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
# 从项目中导入我们需要的模块
from data_manager.loader import load_and_clean_data
from factor_library.fundamental.size_factor import SizeFactor
from backtest_engine.engine import BacktestEngine
from backtest_engine.performance import PerformanceAnalyzer

print(f"项目根目录已设置为: {project_root}")

### 步骤 1: 加载并清洗数据
我们首先调用 `data_manager` 模块中的 `load_and_clean_data` 函数来获取一个干净、对齐的 `master_data` 数据框。这是所有因子计算的基础。

In [ ]:
DATA_DIR_PATH = os.path.join(project_root, "data_manager", "DemoData")

master_df, index_df = load_and_clean_data(DATA_DIR_PATH)

if master_df is not None:
    print("\n主数据加载成功，预览:")
    display(master_df.head())

### 步骤 2: 计算规模因子
我们实例化 `SizeFactor` 类，并将 `master_df` 传入。然后调用 `calculate_factor` 方法来获得因子值。

**理论依据**: 根据《因子投资：方法与实践》，规模因子是经典的股票风格因子之一。通常使用总市值的对数值来代表，这样做可以降低极端值的影响，使数据分布更接近正态分布。

In [ ]:
size_calculator = SizeFactor(master_df)
size_factor_df = size_calculator.calculate_factor()

print("\n规模因子计算完成，预览:")
display(size_factor_df.head())

### 步骤 3: 运行分组回测
接下来，我们将 `master_df` 和计算出的 `size_factor_df` 传入 `BacktestEngine`。在这里，我们需要做出一个关键的假设：
- `long_direction`: 这个参数决定了我们是做多因子值大的一组还是小的一组。
  - `'high'`: 做多因子值最高的组 (Group 5)。
  - `'low'`: 做多因子值最低的组 (Group 1)。

对于规模因子，学术界存在争议。经典的Fama-French三因子模型认为小市值公司有超额收益（应设为`'low'`）。但在A股市场的某些阶段，大盘股表现更优（应设为`'high'`）。**我们先假设大市值股票收益更高，设置为`'high'`，通过回测来验证这个假设。**

In [ ]:
backtest = BacktestEngine(
    master_data=master_df, 
    factor_data=size_factor_df, 
    n_groups=5, 
    long_direction='high' # 假设做多大市值组合
)

portfolio_returns = backtest.run()

print("\n回测完成，各投资组合每日收益率预览：")
display(portfolio_returns.head())

### 步骤 4: 性能分析与可视化
最后，我们使用 `PerformanceAnalyzer` 对回测结果进行全面的分析。它会自动计算年化收益、夏普比率、最大回撤等关键指标，并绘制净值曲线图、分组收益图和IC分析图等。

In [ ]:
if portfolio_returns is not None:
    # 将所有需要的数据传入分析器
    analyzer = PerformanceAnalyzer(portfolio_returns, size_factor_df, master_df)
    
    # 1. 计算各项性能指标
    analyzer.calculate_metrics()
    
    # 2. 绘制图表
    analyzer.plot_results()

### 结论解读

根据输出的图表和指标，我们可以对规模因子在回测期间的表现做出判断：
- **净值曲线 (`Cumulative Returns`)**: `Long_Only`（做多大市值组合）和 `Long_Short`（多空对冲）组合的净值是否长期向上？如果`Long_Short`组合净值持续稳定上升，说明因子具有较好的单调性和选股能力。
- **分组收益 (`Group Cumulative Returns`)**: 从Group 1到Group 5的最终净值是否呈现单调递增或递减的趋势？对于我们`long_direction='high'`的假设，我们期望看到Group 5的收益显著高于其他组，特别是Group 1。
- **IC分析 (`Information Coefficient Analysis`)**: IC代表了因子预测未来收益的能力。IC的均值是否显著大于0？IC的t检验p值是否小于0.05？这些都能量化地告诉我们因子的有效性。

通过这个标准化的流程，我们可以对任何新想到的因子进行快速、严谨的检验。